In [14]:
import pandas as pd
import numpy as np
import io

# Load dataset dummy
csv_data = """
transaction_id,timestamp,user_id,amount,promo_code,status,device_info
TXN-001,2024-01-01 10:00:00,USER_123,Rp 150.000,SALE2024,Success,Android 10
TXN-002,01/01/2024 10:05:00,USER_456,12.50 USD,,Pending,iPhone 12
TXN-003,2024-01-01 10:10:00,USER_789,Rp 500000,CASHBACK50,Failed,Unknown
TXN-004,2024-01-01 10:15:00,USER_123,150000,SALE2024,Success,Android 10
TXN-001,2024-01-01 10:00:00,USER_123,Rp 150.000,SALE2024,Success,Android 10
TXN-005,2024-01-01 12:00:00,,Rp 1.000.000,VIP_PROMO,Success,Windows PC
TXN-006,2024-01-02 08:30:00,USER_999,99.99,Unknown,Success,MacBook Pro
TXN-007,Jan 02 2024 09:00,USER_888,-50000,,Refunded,Android 11
TXN-008,2024-01-02 09:15:00,USER_777,Rp 2.500.000,SUPERDEAL,Success,iPhone 14
TXN-009,2024-01-02 09:15:00,USER_777,Rp 2.500.000,SUPERDEAL,Success,iPhone 14
TXN-010,2024-01-03 14:20:00,USER_123,Rp 150.000,SALE2024,Success,Android 10
TXN-011,2024-01-03 15:00:00,USER_555,N/A,,Pending,Linux
TXN-012,2024-01-04 16:45:00,USER_666,Rp 5.000.000,Unknown,Success,iPhone 13
TXN-013,2024-01-04 17:00:00,USER_456,12.50 USD,SALE2024,Success,iPhone 12
TXN-014,2024-01-05 10:00:00,USER_333,Rp 75000,,Failed,Android 9
TXN-015,2024-01-06 11:00:00,USER_333,Rp 125000,SALE2024,Failed,Android 9
TXN-016,2024-01-06 14:25:00,USER_333,Rp 90000,SALE2024,Failed,Android 9
"""

df = pd.read_csv(io.StringIO(csv_data))
print("Data has been loaded successfully.")
df.head(15)

Data has been loaded successfully.


,transaction_id,timestamp,user_id,amount,promo_code,status,device_info
0,TXN-001,2024-01-01 10:00:00,USER_123,Rp 150.000,SALE2024,Success,Android 10
1,TXN-002,01/01/2024 10:05:00,USER_456,12.50 USD,NaN,Pending,iPhone 12
2,TXN-003,2024-01-01 10:10:00,USER_789,Rp 500000,CASHBACK50,Failed,Unknown
3,TXN-004,2024-01-01 10:15:00,USER_123,150000,SALE2024,Success,Android 10
4,TXN-001,2024-01-01 10:00:00,USER_123,Rp 150.000,SALE2024,Success,Android 10
5,TXN-005,2024-01-01 12:00:00,NaN,Rp 1.000.000,VIP_PROMO,Success,Windows PC
6,TXN-006,2024-01-02 08:30:00,USER_999,99.99,Unknown,Success,MacBook Pro
7,TXN-007,Jan 02 2024 09:00,USER_888,-50000,NaN,Refunded,Android 11
8,TXN-008,2024-01-02 09:15:00,USER_777,Rp 2.500.000,SUPERDEAL,Success,iPhone 14
9,TXN-009,2024-01-02 09:15:00,USER_777,Rp 2.500.000,SUPERDEAL,Success,iPhone 14


In [15]:
# Tampilkan info awal untuk melihat tipe data yang 'rusak'
print("DataFrame Info")
df.info()

DataFrame Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  17 non-null     object
 1   timestamp       17 non-null     object
 2   user_id         16 non-null     object
 3   amount          16 non-null     object
 4   promo_code      13 non-null     object
 5   status          17 non-null     object
 6   device_info     17 non-null     object
dtypes: object(7)
memory usage: 1.1+ KB


In [16]:
# Hapus Duplikat
df = df.drop_duplicates(keep='first').reset_index(drop=True)

# Membersihkan Format Tanggal (Standardisasi Timestamp)
df.loc[df['timestamp'].str.startswith("Jan"), 'timestamp'] = "2024-02-01 09:00:00"
df.loc[df['timestamp'] == '01/01/2024 10:05:00', 'timestamp'] = "2024-01-01 10:05:00"

# Memecah menjadi Date & Time lalu konversi ke datetime object
df[['date', 'time']] = df['timestamp'].str.split(" ", expand=True)
df['date'] = pd.to_datetime(df['date'])

# Pastikan kolom timestamp juga jadi datetime agar bisa diolah
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Menangani Missing Values (User ID & Promo Code)
df = df.dropna(subset=['user_id'])

# Isi Promo Code kosong/Unknown dengan "NO_PROMO"
df['promo_code'] = df['promo_code'].fillna("NO_PROMO")
df.loc[df['promo_code'].str.contains('Unknown', case=False, na=False), 'promo_code'] = "NO_PROMO"

In [17]:
# Deteksi Tipe Mata Uang (IDR vs USD)
mask_rp = df['amount'].astype(str).str.contains("Rp", na=False)
mask_usd = df['amount'].astype(str).str.contains("USD", na=False)

# Konversi USD ke IDR
# Ambil angka depan sebelum kata ' USD', ubah ke float, kali kurs
df.loc[mask_usd, 'amount'] = df.loc[mask_usd, 'amount'].str.split(' ').str[0].astype(float) * 15000

# Membersihkan Format Rupiah (Hapus 'Rp', Titik, Koma)
df.loc[mask_rp, 'amount'] = df.loc[mask_rp, 'amount'].astype(str).str.replace("Rp ", "", regex=False)

# Hapus titik (pemisah ribuan di Indo) dan koma
df['amount'] = df['amount'].astype(str).str.replace('.', '', regex=False).str.replace(',', '', regex=False)

# Menangani Nilai Aneh
df.loc[df['amount'] == '9999', 'amount'] = 99900 

# Menangani 'N/A' atau 'nan' string menjadi NaN object numpy
df['amount'] = df['amount'].replace({'N/A': np.nan, 'nan': np.nan})

# Konversi ke Numerik & Handling Null
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

# Imputasi Nilai Kosong dengan Median
median_amount = df['amount'].median()
df['amount'] = df['amount'].fillna(median_amount)

print("\nData have been cleaned.")
print(df.info())


Data have been cleaned.
<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 15
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  15 non-null     object        
 1   timestamp       15 non-null     datetime64[ns]
 2   user_id         15 non-null     object        
 3   amount          15 non-null     float64       
 4   promo_code      15 non-null     object        
 5   status          15 non-null     object        
 6   device_info     15 non-null     object        
 7   date            15 non-null     datetime64[ns]
 8   time            15 non-null     object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.2+ KB
None


In [18]:
print("\n Analysis Promo Abuse")

# Mencari user yang menggunakan kode 'SALE2024' lebih dari 2 kali
df_sale = df[df['promo_code'] == 'SALE2024']
promo_abusers = df_sale['user_id'].value_counts()
abuser_list = promo_abusers[promo_abusers > 2]

if not abuser_list.empty:
    print(f"Founded {len(abuser_list)} suspicious user (Spam Promo more than 2 times):")
    for user, count in abuser_list.items():
        print(f"- {user}: {count} transaction")
else:
    print("Promo Abuse not founded.")


 Analysis Promo Abuse
Founded 1 suspicious user (Spam Promo more than 2 times):
- USER_123: 3 transaction


In [19]:
print("\nTransaction Summary")

# Hitung jumlah transaksi berdasarkan Status
success_count = df[df['status'] == 'Success'].shape[0]
failed_count = df[df['status'].isin(['Failed', 'Refunded'])].shape[0] # Gabung Failed & Refunded

print(f"Total Successfull Transaction   : {success_count}")
print(f"Total Loss Potential            : {failed_count} (Failed/Refunded)")

# Tampilkan 5 data teratas hasil akhir
print("\nPreview Final Data")
print(df[['transaction_id', 'user_id', 'amount', 'promo_code', 'status']].head())


Transaction Summary
Total Successfull Transaction   : 8
Total Loss Potential            : 5 (Failed/Refunded)

Preview Final Data
  transaction_id   user_id     amount  promo_code   status
0        TXN-001  USER_123   150000.0    SALE2024  Success
1        TXN-002  USER_456  1875000.0    NO_PROMO  Pending
2        TXN-003  USER_789   500000.0  CASHBACK50   Failed
3        TXN-004  USER_123   150000.0    SALE2024  Success
5        TXN-006  USER_999    99900.0    NO_PROMO  Success
